# Colab-CSA (pytorch)
Original repo: [KumapowerLIU/CSA-inpainting](https://github.com/KumapowerLIU/CSA-inpainting)

Yukariin's repo: [Yukariin/CSA_pytorch](https://github.com/Yukariin/CSA_pytorch)
Thank you Yukariin for fixing the broken original.

Differentiable Augmentation: [mit-han-lab/data-efficient-gans](https://github.com/mit-han-lab/data-efficient-gans)

My fork: [styler00dollar/Colab-CSA-pytorch](https://github.com/styler00dollar/Colab-CSA-pytorch)

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/styler00dollar/Colab-CSA-pytorch
!pip install tensorboardX

# Training

Warning: It saves into the folder named ckpt. If you only create save_dir, then it will crash during saving.

In [ ]:
#@title Create empty folders
!mkdir /content/root
!mkdir /content/save_dir
!mkdir /content/save_dir/ckpt

In [ ]:
%cd /content/Colab-CSA-pytorch
!python /content/Colab-CSA-pytorch/train.py --root /content/root --save_dir /content/save_dir

# Testing 

In [ ]:
!python /content/CSA_pytorch/test.py --image /content/image.png \
--mask /content/mask.png --output /content/output.png \
--checkpoint /content/save_dir/ckpt/G_10000.pth